In [38]:
import json
import os
import re

annotations_folder = 'annotations'
annotations_file_1 = 'final_annotations_pedro.json'
annotations_file_2 = 'final_annotations_claudio.json'
final_annotations_file = 'final_annotations.json'

file_path_1 = os.path.join(annotations_folder, annotations_file_1)
file_path_2 = os.path.join(annotations_folder, annotations_file_2)
file_path_3 = os.path.join(annotations_folder, final_annotations_file)

with open(file_path_1, 'r', encoding='utf-8') as f1:
    data_1 = json.load(f1)
f1.close()

with open(file_path_2, 'r', encoding='utf-8') as f2:
    data_2 = json.load(f2)
f2.close()

with open(file_path_3, 'r', encoding='utf-8') as f3:
    final_data = json.load(f3)
f3.close()

In [45]:
def convert_annotations_to_strings(text, annotations, match_length):
    entities = annotations["entities"]
    result = []

    for start, end, label in entities:
        entity_number = []
        start = start-match_length
        end = end-match_length
        entity_text = text[start:end]

        while entity_text and (entity_text[0] == ' ' or entity_text[0] == '.'):
            entity_text = entity_text[1:]
            start +=1

        while entity_text and (entity_text[-1] == ' ' or entity_text[-1] == '.'):
            entity_text = entity_text[:-1]
            end -=1
        entity_number.append(start)
        entity_number.append(end)

        result.append([entity_text, label, entity_number])

    return result

In [16]:
def analysis_function(data_1, data_2):
    result_for_metric_1 = []
    result_for_metric_2 = []
    disagreement_list = []
    for index in range(len(data_1["annotations"])):
        aux_list = []
        if (data_1["annotations"][index] is not None) and (data_2["annotations"][index] is not None):
            text = data_1["annotations"][index][0]
            match_length = len(re.match(r'\d_\d+_', text).group())
            text = re.sub(r'\d_\d+_', '', text)

            entities_1 = data_1["annotations"][index][1]

            rotulation_1 = convert_annotations_to_strings(text, entities_1, match_length)

            entities_2 = data_2["annotations"][index][1]

            rotulation_2 = convert_annotations_to_strings(text, entities_2, match_length)

            if len(rotulation_1) != 0:
                if(len(rotulation_1) > len(rotulation_2)):
                    bigger = rotulation_1
                    smaller = rotulation_2
                else:
                    bigger = rotulation_2
                    smaller = rotulation_1
                
                agreement = 0
                only_one_marked = 0
                different_class = 0         
                different_set = 0

                for entity in bigger:
                    discordancia = False
                    if(entity not in smaller):
                        result_for_metric_1.append(entity[0])
                        result_for_metric_2.append(0)
                        if(smaller):
                            for i in range(len(smaller)):
                                if(entity[2] == smaller[i][2]):
                                    different_class += 1
                                    discordancia = True
                                    smaller.remove(smaller[i])
                                    bigger.remove(entity)
                                    break
                                elif (entity[2][0] == smaller[i][2][0]) and (entity[1] == smaller[i][1]):
                                    different_set += 1
                                    discordancia = True
                                    smaller.remove(smaller[i])
                                    bigger.remove(entity)
                                    break
                        if not discordancia:
                            only_one_marked += 1                          
                    else:
                        agreement +=1
                        smaller.remove(entity)
                        result_for_metric_1.append(entity[0])
                        result_for_metric_2.append(entity[0])
                
                only_one_marked += len(smaller)
                for entity in smaller:
                    result_for_metric_1.append(entity[0])
                    result_for_metric_2.append(0)
                aux_list.append(agreement)
                aux_list.append(only_one_marked)
                aux_list.append(different_class)
                aux_list.append(different_set)
            else:
                aux_list.append(0)
                aux_list.append(0)
                aux_list.append(0)
                aux_list.append(0)

        disagreement_list.append(aux_list)
    return result_for_metric_1, result_for_metric_2, disagreement_list

In [17]:
from sklearn.metrics import cohen_kappa_score

results = analysis_function(data_1, data_2)

y1 = results[0]
y2 = results[1]
print(y1)
print(y2)

print(cohen_kappa_score(y1, y2))

count0 = 0
count1 = 0


for y in y2:
    if y == 0:
        count0 += 1
    else:
        count1 += 1

print("Discordâncias: " + str(count0))
print("Concordâncias: " + str(count1))

['Fitting room management and occupancy monitoring system', 'customer', 'collecting and tracking data of fitting room activity', 'preset business rules', 'assists retail operations with reduction of theft incidents', 'employee compliance rates', 'notifying staff', 'tracking data', 'X-ray sensing apparatus', 'detector array', 'generate a plurality of signals', 'high-clock logic device', 'detector array', 'low-clock logic device', 'high-clock logic device', 'accelerometer', 'low-clock logic device', 'processor', 'high-clock logic device.', 'low-clock logic device', 'receive an acceleration signal', 'determine that the acceleration signal corresponds to a wake gesture', 'determining that the acceleration signal corresponds to the wake gesture', 'send a first power state change signal', 'high-clock logic device', 'a second power state change signal to the high-clock logic device', 'intelligent LED light apparatus', 'intelligent LED light apparatus', 'communication interface', 'processor', 

In [11]:
# Concordância - Só um marcou - Classe Diferente - Conjunto Diferente
for result in results[2]:
    print(result[3])

1
3
2
0
1
6
3
2
5
0
0
3
0
4
0
2
0
0
4
3
2
0
0
1
1
1
0
0
0
0
3
1
2
1
0
3
2
0
0
0
2
2
0
0
0
1
2
1
0
0
2
1
1
2
1
0
0
3
0
0
1
2
0
1
1
1
1
3
1
1
0
2
0
0
0
0
0
0
0
2
1
1
0
1
0
2
0
0
0
0
4
0
1
1
0
1
0
0
0
0
0
1
1
0
1
2
1
0
1
3
0
3
0
0
1
0
1
3
2
0
2
2
3
5
3
2
1
0
2
1
2
1
0
0
0
2
0
0
0
0
0
0
0
2


In [54]:
def analysis_function(final_data):

    dataset_aux = []
    final_rotulation = []
    item_count = 0
    patent_count = 0

    for index in range(len(final_data["annotations"])):
        annotation = final_data["annotations"][index]

        if (annotation is not None) and (annotation[1]['entities'] != []):
            
            patent_count += 1

            text = annotation[0]
            
            entities = annotation[1]

            match_length = len(re.match(r'\d_\d+_', text).group())
            text = re.sub(r'\d_\d+_', '', text)
            text = re.sub(r'\r', '', text)

            word_list = re.findall(r'\w+\s?|[^\w]\s?', text, re.UNICODE)
            
            rotulation = convert_annotations_to_strings(text, entities, match_length)
            final_rotulation.append(rotulation)

            start = 0
            end = 0

            for word in word_list:
                final_word = word.split()[0]
                row = []
                end = start + len(word)-1
                row.append(item_count)
                if(final_word == ';'):
                    row.append(',')
                else:
                    row.append(final_word)
                row.append('O')
                row.append(start)
                row.append(end)
                row.append(patent_count)

                dataset_aux.append(row)
                item_count += 1
                start = start + len(word)
    index = 0
    patent_num_aux = 1
    for word in dataset_aux:
        patent_num = word[5]
        if(patent_num != patent_num_aux):
            index += 1
        for entity in final_rotulation[index]:
            if(word[1] != '.'):
                if (word[3] == entity[2][0] and word[4] <= entity[2][1]):
                    word[2] = 'B-' + entity[1]
                elif (word[3] >= entity[2][0] and word[4] <= entity[2][1]):
                    word[2] = 'I-' + entity[1]
                    break
        patent_num_aux = patent_num

    dataset = [sublist[:3] + sublist[5:] for sublist in dataset_aux]
    return dataset

dataset = analysis_function(final_data)

partition_number = 10
dataset_size = len(dataset)
partition_number_size = dataset_size/partition_number

count = 0
partition = 1
for data in dataset:
    count += 1
    if(count > partition_number_size):
        count = 0
        partition += 1
    data.append(partition)
    if(partition == 8):
        data.append("validation")
    elif(partition > 8):
        data.append("test")
    else:
        data.append("training")
    print(data)

[0, 'Fitting', 'O', 1, 1, 'training']
[1, 'room', 'O', 1, 1, 'training']
[2, 'management', 'O', 1, 1, 'training']
[3, 'and', 'O', 1, 1, 'training']
[4, 'occupancy', 'B-SOFTWARE', 1, 1, 'training']
[5, 'monitoring', 'I-SOFTWARE', 1, 1, 'training']
[6, 'system', 'I-SOFTWARE', 1, 1, 'training']
[7, 'for', 'O', 1, 1, 'training']
[8, 'retail', 'O', 1, 1, 'training']
[9, 'operations', 'O', 1, 1, 'training']
[10, 'having', 'O', 1, 1, 'training']
[11, 'fitting', 'O', 1, 1, 'training']
[12, 'rooms', 'O', 1, 1, 'training']
[13, 'wherein', 'O', 1, 1, 'training']
[14, 'the', 'O', 1, 1, 'training']
[15, 'system', 'O', 1, 1, 'training']
[16, 'if', 'O', 1, 1, 'training']
[17, 'capable', 'O', 1, 1, 'training']
[18, 'of', 'O', 1, 1, 'training']
[19, 'notifying', 'B-FUNCIONALIDADE', 1, 1, 'training']
[20, 'staff', 'I-FUNCIONALIDADE', 1, 1, 'training']
[21, 'of', 'O', 1, 1, 'training']
[22, 'fitting', 'O', 1, 1, 'training']
[23, 'room', 'O', 1, 1, 'training']
[24, 'occupancy', 'O', 1, 1, 'training']
[25,

In [55]:
import csv

headers = ["item", "token", "tag", "patent", "partition", "trainingTest"]

with open('final_dataset.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(headers)
    writer.writerows(dataset)